# Tool Calling

In [2]:
import os
import sys
import random
import json
from pydantic import BaseModel, Field

from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage, FunctionMessage
from langchain_openai import OpenAI, ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_core.tools import tool

from IPython.display import Image, display

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from lib.utils import set_chatgpt_env, set_langsmith_env

In [3]:
set_langsmith_env()
set_chatgpt_env()

In [7]:
llm = ChatOpenAI(model="gpt-4o")

class WeatherApiSchema(BaseModel):
    city: str = Field(..., description="The name of the city")


class HotelsAvailabilitySchema(BaseModel):
    city: str = Field(..., description="The name of the city")
    day: str = Field(..., description="Day of the week to book the hotel")


@tool(args_schema=WeatherApiSchema)
def check_weather(city: str) -> str:
    """Check the weather in a specified city."""
    # Simulated weather check
    weathers = ["sunny", "rainy", "cloudy"]
    return f"The weather in {city} is {random.choice(weathers)}"


@tool(args_schema=HotelsAvailabilitySchema)
def check_hotels(city: str, day: str) -> str:
    """Check hotel availability in a specified city for a given day."""
    # Simulated availability check
    available = random.choice([True, False])
    return f"Hotels in {city} for {day} are {'available' if available else 'fully booked'}"


weather_tool = ToolNode(tools=[check_weather])
hotels_tool = ToolNode(tools=[check_hotels])


tools = [check_weather, check_hotels]
llm_with_tools = llm.bind_tools(tools)

tool_mapping = {
    "check_weather": check_weather,
    "check_hotels": check_hotels
}


async def process_tool_calls(llm_output, messages):
    """Process tool calls and add their responses to messages."""
    # Process each tool call
    for tool_call in llm_output.tool_calls:
        # Get the tool function
        tool = tool_mapping[tool_call.function.name]
        
        # Parse the arguments from JSON string
        args = json.loads(tool_call.function.arguments)
        
        # Call the tool with the arguments
        tool_output = tool(**args)
        
        # Create a new tool message
        new_message = FunctionMessage(
            content=tool_output,
            name=tool_call.function.name,
            additional_kwargs={"tool_call_id": tool_call.id}
        )
        
        # Add the message to the list
        messages.append(new_message)
    
    # Get new LLM response with updated messages
    llm_output = await llm.ainvoke(messages)
    print(llm_output)
    
    return llm_output, messages

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant used foWhat's the weather in Tokyo and are hotels available on Saturday?r vacation planning"),
    HumanMessage(content="What is the weather in Tokyo? I would like to go for a weekend-long hike and book one room for Saturday.")
]

In [10]:
result = llm_with_tools.invoke("")


content='' additional_kwargs={'tool_calls': [{'id': 'call_mdJ4tgHbaVccejz0Ke02jcRt', 'function': {'arguments': '{\n  "city": "Tokyo"\n}', 'name': 'check_weather'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 128, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-de8787bc-c8e4-436f-af16-d69102b968d0-0' tool_calls=[{'name': 'check_weather', 'args': {'city': 'Tokyo'}, 'id': 'call_mdJ4tgHbaVccejz0Ke02jcRt', 'type': 'tool_call'}] usage_metadata={'input_tokens': 128, 'output_tokens': 17, 'total_tokens': 145, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
